In [1]:
import tensorflow as tf

In [2]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [8]:
sess.close()

## A more efficient way to having to repeat sess.run() is to do the following

In [9]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [10]:
init = tf.global_variables_initializer() #prepare an init node

In [11]:
with tf.Session() as sess:
    init.run() # initialize all variables
    result = f.eval()

In [12]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

42


In [13]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

#### Any time a node is created it gets added to the graph. In order to combat and manage this it is possible to create a new graph and temporarily make it the default graph within a block

In [14]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
    
x2.graph is graph

True

In [15]:
x2.graph is tf.get_default_graph()

False

### While troubleshooting or coding our graphs, we may run into a case with many overlapping nodes and issues. 

#### In order to correct this we may take advantage of tf.reset_default_graph()

In [16]:
tf.reset_default_graph()

### In the following code below, x and w will both be evaluated twice for y, and z.

In [17]:
w = tf.constant(3)
x = w + 1
y = x + 6
z = x * 4

with tf.Session() as sess:
    print(y.eval()) 
    print(z.eval())

10
16


## Say if we only want to evaluate x and w once, we may do so in just one graph. 

In [18]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y.eval()) 
    print(z.eval())

10
16


### Utilizing tensorflow for more than a scalar vector....
#### Let's do some computations

In [19]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [20]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

In [22]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)

[[-3.6959320e+01]
 [ 4.3698898e-01]
 [ 9.4245886e-03]
 [-1.0791138e-01]
 [ 6.4842808e-01]
 [-3.9986235e-06]
 [-3.7866351e-03]
 [-4.2142656e-01]
 [-4.3467718e-01]]
